In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv",index_col=0)
df.head(5)

,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01
1,RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV,RAIN's Official Channel,10,2600864,0,0,20129,UCxXgIeE5hxWxHG6dz9Scg2w,2021-01-01
2,2020년 제야의 종 온라인 타종행사 | 보신각 현장 행사는 진행하지 않습니다.,서울시 · Seoul,29,347049,3564,120,178,UCZUPZW5idAxYp-Asj__lVAA,2021-01-01
3,고기남자의 칠면조 파티,고기남자 MeatMan,26,528458,15372,280,3470,UCT3CumbFIJiW33uq0UI3zlg,2021-01-01
4,골목 3mc를 분노하게 만든 마음고생이 심했을 공릉 백반집 사장님의 푸념?! [예능...,스브스밥집,24,494904,3918,111,3142,UCdWgRSfttvDucq4ApcCg5Mw,2021-01-01


In [ ]:
# 인기동영상 제작횟수가 많은 채널 상위 10개명을 출력하라 (날짜기준, 중복포함)
top_channels = df["channelTitle"].value_counts(ascending=False).index.tolist() # value_counts로 인덱스로 들어간 채널명을 가져와 리스트로 변경
top_channels
# 논란으로 인기동영상이 된 케이스를 확인하고 싶다. dislikes수가 like 수보다 높은 동영상을 제작한 채널을 모두 출력하라
df.query('dislikes > likes')['channelTitle'].unique() # 쿼리 함수, 고유값찾기
# 채널명을 바꾼 케이스가 있는지 확인하고 싶다. channelId의 경우 고유값이므로 이를 통해 채널명을 한번이라도 바꾼 채널의 갯수를 구하여라
change = df.groupby('channelId')['channelTitle'].nunique() # 채널id별 채널명의 고유값 갯수 가져오고 채널id가 2개 이상인 값 추출, 그 값의 length
change_channel = change[change > 1]
len(change_channel)
# 일요일에 인기있었던 영상들중 가장많은 영상 종류(categoryId)는 무엇인가?
df['trending_date'] = pd.to_datetime(df['trending_date2'], format='%Y-%m-%d') # sql에 TO_DATE와 유사한 기능 yyyy면 y가 대문자
sunday_df = df[df['trending_date'].dt.weekday == 6] # dt.weekday()는 0이 월요일, 6이 일요일 tending_date가 dt.weekday 6 인 값들
sundday_categoryId = sunday_df['categoryId'].value_counts() # 6인 값들의 채널id들
sundday_categoryId.head(1)
# 각 요일별 인기 영상들의 categoryId는 각각 몇개 씩인지 하나의 데이터 프레임으로 표현하라
df['tranding_date'] = pd.to_datetime(df['trending_date2'], format='%Y-%m-%d') # pd.to_datetime으로 변환
df['day_name'] = df['tranding_date'].dt.day_name() # dt.day_name으로 요일이름 가져오기
group = df.groupby(['day_name', 'categoryId']).size().reset_index(name="count")# 요일이름 채널 아이디의 사이즈 출력하고 인덱스로 바꾼뒤 인덱스 초기화 후 count 컬럼 생성
pivot_table = group.pivot(index='day_name', columns='categoryId', values='count') # 피봇으로 인덱스,컬럼,값 지정
pivot_table.transpose() # 채널id와 요일이름 위치변경
# 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다. viewcount대비 댓글수가 가장 높은 영상을 확인하라 (view_count값이 0인 경우는 제외한다)
zero = df.loc[df['view_count'] != 0] # 조회수가 0인 값 제외
comment = zero['comment_count'] / zero['view_count'] # 비율 계산 
best_index = comment.sort_values(ascending=False).index[0] # view카운트 대비 댓글수가 가장 높은 영상의 채널의 인덱스
result = df.loc[best_index, 'title'] # 해당 인덱스의 제목 출력
result
# 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다.viewcount대비 댓글수가 가장 낮은 영상을 확인하라 (view_counts, ratio값이 0인경우는 제외한다.)
zero = df.loc[df['view_count'] != 0]
comment = zero['comment_count'] / zero['view_count']
ratio = comment[comment != 0] # 비율 계산 후 0이 아닌값
min_ratio = ratio.idxmin() # 0이 아닌 값중에 제일 낮은 값 idxmin() 인덱스 중 최소값
title = zero.loc[min_ratio, 'title'] 
title
# like 대비 dislike의 수가 가장 적은 영상은 무엇인가? (like, dislike 값이 0인경우는 제외한다)
zero = df.loc[(df['likes'] != 0) & (df['dislikes'] != 0),['likes', 'dislikes', 'title']] # loc의 and 연산 0제외
ratio = zero['dislikes'] / zero['likes'] # 비율 계산
min_likes = ratio.idxmin() # 인덱스 최소값
title = zero.loc[min_likes,'title'] # 인덱스 최소값 영상의 제목
title
# 가장많은 트렌드 영상을 제작한 채널의 이름은 무엇인가? (날짜기준, 중복포함)
best_create = df['channelId'].value_counts().index[0]  # 채널id가 가장 많은 인덱스
title = df.loc[df['channelId']==best_create,'channelTitle'].unique() # id가 많은, 영사잉 많은 채널 id의 채널 이름 , 중복제거
title
# 20회(20일)이상 인기동영상 리스트에 포함된 동영상의 숫자는?
best_video = (df[['title','channelTitle']].value_counts() >= 20).sum()
best_video

40

In [ ]:
top_view= df['view_count'].head(10).sort_values(ascending=False).index

top_view_rows = df.loc[top_view]
display(top_view_rows)

,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2,trending_date,tranding_date,day_name
1,RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV,RAIN's Official Channel,10,2600864,0,0,20129,UCxXgIeE5hxWxHG6dz9Scg2w,2021-01-01,2021-01-01,2021-01-01,Friday
7,[아이유의 팔레트] 팔레트에 '있지' (With ITZY) Ep.4,이지금 [IU Official],24,2420537,200301,956,14641,UC3SyT4_WLHzN7JmHQwKQZww,2021-01-01,2021-01-01,2021-01-01,Friday
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01,2021-01-01,2021-01-01,Friday
9,나 숙취 있어! 알딸딸해! 필름 끊겼어를 영어로 어떻게 할까요?,Sarah Kim,27,1358098,8224,820,1406,UCMXFwld5hCpMc2-49cndljA,2021-01-01,2021-01-01,2021-01-01,Friday
5,"(EN/CN/ID) 월드스타 한 분 더 모셨습니다. 비x박진영 & 싸이, 비주얼 가...",시즌비시즌 Season B Season,24,1274464,26660,368,2701,UCZf4ZESHAIuRtZ-eoMSL97A,2021-01-01,2021-01-01,2021-01-01,Friday
8,답답한 집콕.. 2020년은 예능으로 존버(존중하며 버팀)했다 MBC 연예대상 하이...,14F 일사에프,26,1001581,9462,337,1684,UCLKuglhGlMmDteQKoniENIQ,2021-01-01,2021-01-01,2021-01-01,Friday
6,[미스트롯2] 많은 참가자가 라이벌로 뽑은 명실상부 트롯 천재! 전유진이 부르는 '...,TV CHOSUN JOY,24,682608,7435,434,1943,UCYeeEwH_s9yD9HQe4e2JuBw,2021-01-01,2021-01-01,2021-01-01,Friday
3,고기남자의 칠면조 파티,고기남자 MeatMan,26,528458,15372,280,3470,UCT3CumbFIJiW33uq0UI3zlg,2021-01-01,2021-01-01,2021-01-01,Friday
4,골목 3mc를 분노하게 만든 마음고생이 심했을 공릉 백반집 사장님의 푸념?! [예능...,스브스밥집,24,494904,3918,111,3142,UCdWgRSfttvDucq4ApcCg5Mw,2021-01-01,2021-01-01,2021-01-01,Friday
2,2020년 제야의 종 온라인 타종행사 | 보신각 현장 행사는 진행하지 않습니다.,서울시 · Seoul,29,347049,3564,120,178,UCZUPZW5idAxYp-Asj__lVAA,2021-01-01,2021-01-01,2021-01-01,Friday


In [ ]:
import pandas as pd


channel =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv')
video =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')
display(channel.head())
display(video.head())

,channelid,subcnt,viewcnt,videocnt,ct,channelname
0,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,410238653,736,2021-09-30 03:01:03,꽈뚜룹
1,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,736,2021-09-30 09:01:03,꽈뚜룹
2,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,735,2021-09-30 15:01:03,꽈뚜룹
3,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,737,2021-09-30 21:01:03,꽈뚜룹
4,UCkQCwnkQfgSuPTTnw_Y7v7w,1320000,412531322,737,2021-10-01 03:01:04,꽈뚜룹


,videopk,viewcnt,likecnt,dislikecnt,favoritecnt,cmcnt,ct,videoname
0,c5JQp6xafqc,1667010,30474,706,0,6587,2021-10-10 15:20:03,공범 EP1
1,c5JQp6xafqc,1669089,30495,707,0,6589,2021-10-10 15:30:03,공범 EP1
2,c5JQp6xafqc,1674759,30522,711,0,6596,2021-10-10 15:40:02,공범 EP1
3,c5JQp6xafqc,1677026,30555,712,0,6604,2021-10-10 15:50:03,공범 EP1
4,c5JQp6xafqc,1681824,30585,713,0,6600,2021-10-10 16:00:03,공범 EP1


In [ ]:
# 각 데이터의 ‘ct’컬럼을 시간으로 인식할수 있게 datatype을 변경하고 video 데이터의 videoname의 각 value 마다 몇개의 데이터씩 가지고 있는지 확인하라
# channel['ct'] = pd.to_datetime(channel['ct']) # to_datetime
# # channel['ct'].dtypes
# video['videoname'].value_counts()
# 수집된 각 video의 가장 최신화 된 날짜의 viewcount값을 출력하라
# ans2 = video[['viewcnt','videoname','ct']].sort_values(by='viewcnt', ascending=False)\
#     .drop_duplicates(subset='videoname', keep='first')\
#     .reset_index(drop=True) # 3컬럼을 가져와 viewcnt 기준으로 내림차순 정렬 후 videoname의 중복제거,첫번째 값만 남김 인덱스 초기화 후 인덱스 컬럼 삭제
# ans2
# Channel 데이터중 2021-10-03일 이후 각 채널의 처음 기록 됐던 구독자 수(subcnt)를 출력하라
# channel['ct'] = pd.to_datetime(channel['ct'])  # ct를 datetime으로 변환
# channel_after = channel[channel['ct'] >= '2021-10-03'] # 2021-10-03 이후 데이터만 필터링
# channel_sorted = channel_after.sort_values(by='ct') # ct 기준으로 오름차순 정렬 (먼저 수집된 게 위로)
# channel_first = channel_sorted.drop_duplicates(subset='channelname', keep='first') # 채널마다 처음 나오는 값만 남김
# ans3 = channel_first[['channelname', 'subcnt']].reset_index(drop=True) # 컬럼 정리
# ans3
# 각채널의 2021-10-03 03:00:00 ~ 2021-11-01 15:00:00 까지 구독자수 (subcnt) 의 증가량을 구하여라
# channel['ct'] = pd.to_datetime(channel['ct']) # 날짜 형식 변환
# channel_range = channel[(channel['ct'] >= '2021-10-03 03:00:00') &  (channel['ct'] <= '2021-11-01 15:00:00')]  # 해당 날짜 범위로 필터링
# channel_sorted = channel_range.sort_values(by='ct') # 시간순 정렬
# grouped = channel_sorted.groupby('channelname')['subcnt'] # 채널별 구독자 수
# first = grouped.first() # 제일 먼저 구한 값, 내림차순으로 맨 아래값 시간순으로 처음, 옛날 구독자 수
# last = grouped.last() # 마지막 구독자 수, 최근 구독자 수
# increase = last - first # 구독자 증가량 계산 (마지막 - 처음)
# ans4 = pd.DataFrame({
#     'channelname': increase.index,
#     'del': increase.values
# }).reset_index(drop=True)
# ans4
# 각 비디오는 10분 간격으로 구독자수, 좋아요, 싫어요수, 댓글수가 수집된것으로 알려졌다. 공범 EP1의 비디오정보 데이터중 수집간격이 5분 이하, 20분이상인 데이터 구간( 해당 시점 전,후) 의 시각을 모두 출력하라
# video['ct'] = pd.to_datetime(video['ct'])
# ep1 = video[video['videoname'].str.strip() == '공범 EP1'] # 공범 EP1 가져옴
# ep1 = ep1.sort_values(by='ct') # 시간 순 정렬
# ep1['prev_ct'] = ep1['ct'].shift(1) # 이전 시간 만들기 (한 행씩 밀기)
# ep1['gap_min'] = (ep1['ct'] - ep1['prev_ct']).dt.total_seconds() / 60 # 시간 차이 계산 (분 단위)
# ans5 = ep1[(ep1['gap_min'] <= 5) | (ep1['gap_min'] >= 20)]# 5분 이하 또는 20분 이상인 구간만 추출
# ans5 = ans5[video.columns]  # 원래 컬럼 순서로만 출력
# ans5
# 각 에피소드의 시작날짜(년-월-일)를 에피소드 이름과 묶어 데이터 프레임으로 만들고 출력하라
# video['ct'] = pd.to_datetime(video['ct'])
# video_sorted = video.sort_values(by='ct') # 시간 순으로 정렬
# video_first = video_sorted.drop_duplicates(subset='videoname', keep='first') # 에피소드 이름마다 가장 처음값만 남기기
# video_first['date'] = video_first['ct'].dt.date # 날짜만 추출 (yyyy-mm-dd)
# ans6 = video_first[['date', 'videoname']]
# ans6
# “공범” 컨텐츠의 경우 19:00시에 공개 되는것으로 알려져있다. 공개된 날의 21시의 viewcnt, ct, videoname 으로 구성된 데이터 프레임을 viewcnt를 내림차순으로 정렬하여 출력하라
# video['ct'] = pd.to_datetime(video['ct'])
# video['hour'] = video['ct'].dt.hour # 시간에서 '시(hour)'만 추출
# video_21 = video[(video['videoname'].str.contains('공범')) & (video['hour'] == 21)]\
# .drop_duplicates(subset='videoname', keep='first') # '공범' 포함 + 21시에 수집된 데이터만 선택, videoname 중복제거
# ans7 = video_21[['videoname', 'viewcnt', 'ct']]\
#     .sort_values(by='viewcnt', ascending=False).reset_index(drop=True) # 필요한 컬럼만 정리하고 조회수 내림차순 정렬
# ans7
# video 정보의 가장 최근 데이터들에서 각 에피소드의 싫어요/좋아요 비율을 ratio 컬럼으로 만들고 videoname, ratio로 구성된 데이터 프레임을 ratio를 오름차순으로 정렬하라
# video['ct'] = pd.to_datetime(video['ct'])
# video_sorted = video.sort_values(by='ct', ascending=False) # 시간 순서로 내림차순 정렬
# video_latest = video_sorted.drop_duplicates(subset='videoname', keep='first') # 각 에피소드에서 가장 최근 데이터 하나씩만 남기기
# video_latest['ratio'] = video_latest['dislikecnt'] / video_latest['likecnt'] # 비율 계산
# ans8 = video_latest[['videoname', 'ratio']].sort_values(by='ratio').reset_index(drop=True) # 필요한 컬럼만 정리하고 오름차순 정렬
# ans8
# 2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량을 데이터 프레임으로 만드시오
# video['ct'] = pd.to_datetime(video['ct'])
# video_range = video[(video['ct'] >= '2021-11-01 00:00:00') & (video['ct'] <= '2021-11-01 15:00:00')] # 원하는 시간 범위로 필터링
# video_sorted = video_range.sort_values(by='ct') # 시간 순으로 정렬
# grouped = video_sorted.groupby('videoname')['viewcnt'] # 이름별 조회수
# first = grouped.first()
# last = grouped.last()
# increase = last - first # 증가량 계산
# ans9 = pd.DataFrame({
#     'viewcnt': increase
# })
# ans9
# video 데이터 중에서 중복되는 데이터가 존재한다. 중복되는 각 데이터의 시간대와 videoname 을 구하여라
# video['ct'] = pd.to_datetime(video['ct'])
# duplicates = video[video.duplicated()] # 완전히 같은 행이 중복된 것 찾기
# ans10 = duplicates[['videoname', 'ct']]
# ans10

,viewcnt
videoname,
공범 EP1,13298
공범 EP2,10300
공범 EP3,9927
공범 EP4,9824
공범 EP5,10824
공범 EP6,14141
공범 EP7,26949
공범 EP8,89147


In [236]:
import pandas as pd

df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/worldcup/worldcupgoals.csv')
df

,Player,Goals,Years,Country
0,Miroslav Klose,16,2002-2006-2010-2014,Germany
1,Ronaldo,15,1998-2002-2006,Brazil
2,Gerd Muller,14,1970-1974,Germany
3,Just Fontaine,13,1958,France
4,Pele,12,1958-1962-1966-1970,Brazil
...,...,...,...,...
1290,Josip Skoblar,1,1962,Yugoslavia
1291,Safet Susic,1,1982-1990,Yugoslavia
1292,Aleksandar Tirnanic,1,1930,Yugoslavia
1293,Djordje Vujadinovic,1,1930,Yugoslavia


In [ ]:
# 주어진 전체 기간의 각 나라별 골득점수 상위 5개 국가와 그 득점수를 데이터프레임형태로 출력하라
# df['Goals'] = df['Goals'].fillna(0)
# # df['Goals'].isna().sum()
# goals = df.groupby('Country')['Goals'].sum()
# ans1 = goals.sort_values(ascending=False).head(5).to_frame() # 득점 수 기준으로 내림차순 정렬 후 상위 5개 추출, DataFrame형식변환
# ans1
# 주어진 전체기간동안 골득점을 한 선수가 가장 많은 나라 상위 5개 국가와 그 선수 숫자를 데이터 프레임 형식으로 출력하라
# goals = df[df['Goals'] > 0] # 먼저 Goals > 0인 데이터만 추춣
# goals_country = goals.groupby('Country')['Player'].nunique() # 나라별로 득점한 선수 수 세기 선수 중복 제거
# ans2 = goals_country.sort_values(ascending=False).head(5).to_frame() # 내림차순 정렬 후 상위 5개 추출, DataFrame형식변환
# ans2
# Years 컬럼은 년도 -년도 형식으로 구성되어있고, 각 년도는 4자리 숫자이다. 년도 표기가 4자리 숫자로 안된 케이스가 존재한다. 해당 건은 몇건인지 출력하라
df['years'] = df['Years'].str.split('-')
find = df['years'].apply(lambda x: any((not y.isdigit()) or (len(y) != 4) for y in x)) # split으로 나눈 year가 4자리가 아닌 값
# any() 안에 하나라도 true가 있으면 true 반환 str.isdigit() 문자가 하나라도 있으면 False 반환, 공백,마이너스,마침표가 들어가도 False, 여기선 not y니까 True
ans3 = find.sum()
ans3

# **Q3에서 발생한 예외 케이스를 제외한 데이터프레임을 df2라고 정의하고 데이터의 행의 숫자를 출력하라 (아래 문제부터는 df2로 풀이하겠습니다) **
# 월드컵 출전횟수를 나타내는 ‘LenCup’ 컬럼을 추가하고 4회 출전한 선수의 숫자를 구하여라
# Yugoslavia 국가의 월드컵 출전횟수가 2회인 선수들의 숫자를 구하여라
# 2002년도에 출전한 전체 선수는 몇명인가?
# 이름에 ‘carlos’ 단어가 들어가는 선수의 숫자는 몇 명인가? (대, 소문자 구분 x)
# 월드컵 출전 횟수가 1회뿐인 선수들 중에서 가장 많은 득점을 올렸던 선수는 누구인가?
# 월드컵 출전횟수가 1회 뿐인 선수들이 가장 많은 국가는 어디인가?

45

In [ ]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv')
df.head()

In [212]:
# 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜를 출력하라
# 각 일자의 요일을 표기하고 (‘Monday’ ~’Sunday’) ‘day_name’컬럼을 추가하고 이를 이용하여 각 요일별 이용 횟수의 총합을 데이터 프레임으로 출력하라
# 각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력하라
# 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대를 확인하라. 일일권의 경우 일일권 과 일일권(비회원)을 모두 포함하라
# 연령대별 평균 이동거리를 구하여라
# 연령대 코드가 20대인 데이터를 추출하고,이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출한다.최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하여라
# 6월 7일 ~10대의 “이용건수”의 중앙값은?
# 평일 (월~금) 출근 시간대(오전 6,7,8시)의 대여소별 이용 횟수를 구해서 데이터 프레임 형태로 표현한 후 각 대여시간별 이용 횟수의 상위 3개 대여소와 이용횟수를 출력하라
# 이동거리의 평균 이상의 이동거리 값을 가지는 데이터를 추출하여 추출데이터의 이동거리의 표본표준편차 값을 구하여라
# 남성(‘M’ or ‘m’)과 여성(‘F’ or ‘f’)의 이동거리값의 평균값을 구하여라

In [ ]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv',encoding='utf-8')
df.head()

In [ ]:
# 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다. 각년도의 행복랭킹 10위를 차지한 나라의 행복점수의 평균을 구하여라
# 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다. 각년도의 행복랭킹 50위이내의 나라들의 각각의 행복점수 평균을 데이터프레임으로 표시하라
# 2018년도 데이터들만 추출하여 행복점수와 부패에 대한 인식에 대한 상관계수를 구하여라
# 2018년도와 2019년도의 행복랭킹이 변화하지 않은 나라명의 수를 구하여라
# 2019년도 데이터들만 추출하여 각변수간 상관계수를 구하고 내림차순으로 정렬한 후 상위 5개를 데이터 프레임으로 출력하라. 컬럼명은 v1,v2,corr으로 표시하라
# 각 년도별 하위 행복점수의 하위 5개 국가의 평균 행복점수를 구하여라
# 2019년 데이터를 추출하고 해당데이터의 상대 GDP 평균 이상의 나라들과 평균 이하의 나라들의 행복점수 평균을 각각 구하고 그 차이값을 출력하라
# 각년도의 부패에 대한인식을 내림차순 정렬했을때 상위 20개 국가의 부패에 대한인식의 평균을 구하여라
# 2018년도 행복랭킹 50위 이내에 포함됐다가 2019년 50위 밖으로 밀려난 국가의 숫자를 구하여라
# 2018년,2019년 모두 기록이 있는 나라들 중 년도별 행복점수가 가장 증가한 나라와 그 증가 수치는?

In [ ]:
import pandas as pd
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/consum/Tetuan%20City%20power%20consumption.csv')
df.head()

In [ ]:
# DateTime컬럼을 통해 각 월별로 몇개의 데이터가 있는지 데이터 프레임으로 구하여라
# 3월달의 각 시간대별 온도의 평균들 중 가장 낮은 시간대의 온도를 출력하라
# 3월달의 각 시간대별 온도의 평균들 중 가장 높은 시간대의 온도를 출력하라
# Zone 1 Power Consumption 컬럼의 value값의 크기가 Zone 2  Power Consumption 컬럼의 value값의 크기보다 큰 데이터들의 Humidity의 평균을 구하여라
# 각 zone의 에너지 소비량의 상관관계를 구해서 데이터 프레임으로 표기하라
# Temperature의 값이 10미만의 경우 A, 10이상 20미만의 경우 B,20이상 30미만의 경우 C, 그 외의 경우 D라고 할때 각 단계의 데이터 숫자를 구하여라
# 6월 데이터중 12시의 Temperature의 표준편차를 구하여라
# 6월 데이터중 12시의 Temperature의 분산을 구하여라
# Temperature의 평균이상의 Temperature의 값을 가지는 데이터를 Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값은?
# **Temperature의 중간값 이상의 Temperature의 값을 가지는 데이터를Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값은? **

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv')
df.head()

In [ ]:
# Legendary 컬럼은 전설포켓몬 유무를 나타낸다.전설포켓몬과 그렇지 않은 포켓몬들의 HP평균의 차이를 구하여라
# Type 1은 주속성 Type 2 는 부속성을 나타낸다. 가장 많은 부속성 종류는 무엇인가?
# 가장 많은 Type 1 의 종의 평균 Attack 을 평균 Defense로 나눈값은?
# 포켓몬 세대(Generation) 중 가장많은 Legendary를 보유한 세대는 몇세대인가?
# ‘HP’, ‘Attack’, ‘Defense’, ‘Sp. Atk’, ‘Sp. Def’, ‘Speed’ 간의 상관 계수중 가장 절댓값이 큰 두 변수와 그 값을 구하여라
# 각 Generation의 Attack으로 오름차순 정렬시 상위 3개 데이터들(18개)의 Attack의 전체 평균을 구하여라
# 각 Generation의 Attack으로 내림차순 정렬시 상위 5개 데이터들(30개)의 Attack의 전체 평균을 구하여라
# 가장 흔하게 발견되는 (Type1 , Type2) 의 쌍은 무엇인가?
# 한번씩만 존재하는 (Type1 , Type2)의 쌍의 갯수는 몇개인가?
# 한번씩만 존재하는 (Type1 , Type2)의 쌍을 각 세대(Generation)은 각각 몇개씩 가지고 있는가?

In [ ]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/body/body.csv')
df.head()

In [ ]:
# 전체데이터의 수축기혈압(최고) - 이완기혈압(최저)의 평균을 구하여라
# 50~59세의 신장평균을 구하여라
# 연령대 (20~29 : 20대 …) 별  인원수를 구하여라
# 연령대 (20~29 : 20대 …) 별 등급의 숫자를 데이터 프레임으로 표현하라
# 남성 중 A등급과 D등급의 체지방률 평균의 차이(큰 값에서 작은 값의 차)를 구하여라
# 여성 중 A등급과 D등급의 체중의 평균의 차이(큰 값에서 작은 값의 차)를 구하여라
# bmi는 자신의 몸무게(kg)를 키의 제곱(m)으로 나눈값이다. 데이터의 bmi 를 구한 새로운 컬럼을 만들고 남성의 bmi 평균을 구하여라
# bmi보다 체지방율이 높은 사람들의 체중평균을 구하여라
# 남성과 여성의 악력 평균의 차이를 구하여라
# 남성과 여성의 교차윗몸일으키기 횟수의 평균의 차이를 구하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/weather/weather2.csv")
df.head()

In [ ]:
# 여름철(6월,7월,8월) 이화동이 수영동보다 높은 기온을 가진 시간대는 몇개인가?
# 이화동과 수영동의 최대강수량의 시간대를 각각 구하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/train.csv")
df.head()

In [ ]:
# 남성 이탈(Exited)이 가장 많은 국가(Geography)는 어디이고 이탈 인원은 몇명인가?
# **카드를 소유(HasCrCard ==1)하고 있으면서 활성멤버(IsActiveMember ==1) 인 고객들의 평균 나이를 소숫점이하 4자리까지 구하여라? **
# Balance 값이 중간값 이상을 가지는 고객들의 CreditScore의 표준편차를 소숫점이하 3자리까지 구하여라


In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/train.csv")
df.head()

In [ ]:
# 수축기혈압과 이완기 혈압기 수치의 차이를 새로운 컬럼(‘혈압차’) 으로 생성하고, 연령대 코드별 각 그룹 중 ‘혈압차’ 의 분산이 5번째로 큰 연령대 코드를 구하여라
# 비만도를 나타내는 지표인 WHtR는 허리둘레 / 키로 표현한다. 일반적으로 0.58이상이면 비만으로 분류한다. 데이터중 WHtR 지표상 비만인 인원의 남/여 비율을 구하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/train.csv")
df.head()

In [ ]:
# Vehicle_Age 값이 2년 이상인 사람들만 필터링 하고 그중에서 Annual_Premium 값이 전체 데이터의 중간값 이상인 사람들을 찾고, 그들의 Vintage값의 평균을 구하여라
# vehicle_age에 따른 각 성별(gender)그룹의 Annual_Premium값의 평균을 구하여 아래 테이블과 동일하게 구현하라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/mobile/train.csv")
df.head()

In [ ]:
# price_range 의 각 value를 그룹핑하여 각 그룹의 n_cores 의 빈도가 가장높은 value와 그 빈도수를 구하여라
# price_range 값이 3인 그룹에서 상관관계가 2번째로 높은 두 컬럼과 그 상관계수를 구하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/train.csv")
df.head()

In [ ]:
# Arrival Delay in Minutes 컬럼이 결측치인 데이터들 중 ‘neutral or dissatisfied’ 보다 ‘satisfied’의 수가 더 높은 Class는 어디 인가?

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/train.csv")
df.head()

In [ ]:
# ph값은 상당히 많은 결측치를 포함한다. 결측치를 제외한 나머지 데이터들 중 사분위값 기준 하위 25%의 값들의 평균값은?

In [ ]:
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/train.csv")
train.head()

In [ ]:
# 흡연자와 비흡연자 각각 charges의 상위 10% 그룹의 평균의 차이는?

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/kingcountyprice//train.csv")
df.head()

In [ ]:
# bedrooms 의 빈도가 가장 높은 값을 가지는 데이터들의 price의 상위 10%와 하위 10%값의 차이를 구하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv")
df.head()

In [ ]:
# Serial No. 컬럼을 제외하고 ‘Chance of Admit’을 종속변수, 나머지 변수를 독립변수라 할때, 랜덤포레스트를 통해 회귀 예측을 할 떄 변수중요도 값을 출력하라 (시드값에 따라 순서는 달라질수 있음)

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/train.csv")
df.head()

In [ ]:
# quality 값이 3인 그룹과  8인 데이터그룹의 각 컬럼별 독립변수의 표준편차 값의 차이를 구할때 그값이 가장 큰 컬럼명을 구하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/drug/train.csv")
df.head()

In [ ]:
# 남성들의 연령대별 (10살씩 구분 0~9세 10~19세 …) Na_to_K값의 평균값을 구해서 데이터 프레임으로 표현하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/train.csv")
df.head()

In [ ]:
# 데이터의 Risk 값에 따른 score_a와 score_b의 평균값을 구하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/train.csv")
df.head()

In [ ]:
# pose값에 따른 각 motion컬럼의 중간값의 가장 큰 차이를 보이는 motion컬럼은 어디이며 그값은?

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/train.csv")
df.head()

In [ ]:
# 정보(row수)가 가장 많은 상위 3차종의 price값의 각 평균값은?

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/train.csv")
df.head()

In [ ]:
# Outcome 값에 따른 각 그룹의  각 컬럼의 평균 차이를 구하여라

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/nflx/NFLX.csv")
df.head()

In [ ]:
# 매년 5월달의 open가격의 평균값을 데이터 프레임으로 표현하라

In [ ]:
pd.set_option('display.max_columns',50)
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/nba/nba.csv",encoding='latin',sep=';')
df.head()

In [ ]:
# Tm 컬럼은 각 팀의 이름을 의미한다. TOR팀의 평균나이를 소수 4째 자리까지 구하여라
# Pos 컬럼은 포지션을 의미한다. 전체 선수 중 최소나이대의 선수들을 필터하고 그들 중 가장 많은 포지션은 무엇인지 확인하라
# 선수들의 이름은 first_name+ 공백 + last_name으로 이루어져 있다. 가장 많은 first_name은 무엇이며 몇 회 발생하는지 확인하라
# PTS컬럼은 경기당 평균득점수 이다. 각포지션별로 경기당 평균득점수의 평균을 구하여라
# PTS컬럼은 경기당 평균득점수 이다. 각포지션별로 경기당 평균득점수의 평균을 구하여라
# G컬럼은 참여한 경기의 숫자이다. 각 팀별로 가장 높은 경기참여 수를 가진 선수들의 경기 참여 숫자의 평균을 구하여라
# Tm의 값이 MIA이며 Pos는 C또는 PF인 선수의 MP값의 평균은?
# 전체 데이터중 G의 평균값의 1.5배 이상인 데이터들만 추출했을때 3P값의 평균은?
# Age의 평균 이상인 그룹과 평균 미만인 그룹간의 G값의 평균의 차이는?
# 평균나이가 가장 젊은 팀은 어디인가
# Pos그룹별 평균 MP값을 구하여라